# Zwei-Stufen-XGBoost – Experimentsteuerung

In diesem Notebook möchte ich Experimente so aufsetzen, dass **alle Daten und Ergebnisse automatisch
mit einer Experiment-ID versioniert werden**.

Grundidee:

- Ich definiere am Anfang eine `EXP_ID` (z.\u00a0B. `v1_h4_thr0p5pct_strict`).
- Daraus werden automatisch erzeugt:
  - `data/processed/fx/eurusd_labels__<EXP_ID>.csv`
  - `data/processed/datasets/eurusd_news_training__<EXP_ID>.csv`
  - sowie jeweils die aktuelle Version ohne Suffix (`..._latest`).
- Sp\u00e4tere Modell-Notebooks k\u00f6nnen diese Dateien direkt verwenden und Ergebnisse als
  `notebooks/results/two_stage__<EXP_ID>.json` speichern.

Damit ist jederzeit klar dokumentiert, **mit welchen Label-Parametern und Features ein Experiment
durchgef\u00fchrt wurde**, und ich kann sp\u00e4ter leicht vergleichen.


In [44]:
import sys
from pathlib import Path

# Aktuelles Arbeitsverzeichnis des Kernels ermitteln.
cwd = Path.cwd()
print("Aktuelles Arbeitsverzeichnis:", cwd)

# Projektwurzel automatisch finden, indem wir nach oben laufen,
# bis ein Ordner `src` existiert.
project_root = cwd
while not (project_root / "src").is_dir():
    project_root = project_root.parent

print("Erkannte Projektwurzel:", project_root)

# Projektwurzel in den PYTHONPATH aufnehmen, damit `import src....` funktioniert.
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))


Aktuelles Arbeitsverzeichnis: /Users/jeremynathan/Documents/GitHub/hs2025_ml_project/hs2025_ml_project
Erkannte Projektwurzel: /Users/jeremynathan/Documents/GitHub/hs2025_ml_project/hs2025_ml_project


In [45]:
import os
from pathlib import Path

# Diese Zelle sorgt dafür, dass das aktuelle Arbeitsverzeichnis (CWD)
# die Projektwurzel ist. Dann zeigt `data/raw/fx/EURUSDX.csv`
# auf die richtige Datei.

print("Arbeitsverzeichnis vor Änderung:", Path.cwd())

# Arbeitsverzeichnis explizit auf die erkannte Projektwurzel setzen.
os.chdir(project_root)

print("Arbeitsverzeichnis nach Änderung:", Path.cwd())


Arbeitsverzeichnis vor Änderung: /Users/jeremynathan/Documents/GitHub/hs2025_ml_project/hs2025_ml_project
Arbeitsverzeichnis nach Änderung: /Users/jeremynathan/Documents/GitHub/hs2025_ml_project/hs2025_ml_project


## 1. Experiment-ID und Label-Parameter definieren

In dieser Zelle lege ich fest:

- `EXP_ID`: eine eindeutige Bezeichnung f\u00fcr dieses Experiment.
- `LABEL_PARAMS`: alle Parameter, die die Label-Logik bestimmen (Horizont, Schwellen, Monotonie).
- `OVERSAMPLE_FACTOR`: sp\u00e4ter relevant f\u00fcr das Oversampling der `signal=1`-F\u00e4lle im Training.

Die Idee ist, dass jede Kombination dieser Parameter als eigene Version gespeichert wird.


In [46]:
# Experiment-Konfiguration
EXP_ID = "s3b_h4_thr0p3pct_tol0p3_30dfeat"  # frei w\u00e4hlbarer Name f\u00fcr dieses Experiment

LABEL_PARAMS = dict(
    horizon_days=4,
    up_threshold=0.003,    # +0.5 % Lookahead-Return \u00fcber 4 Tage
    down_threshold=-0.003, # -0.5 % Lookahead-Return \u00fcber 4 Tage
    strict_monotonic=False, # Pfad t..t+4 muss streng steigend/fallend sein
    max_adverse_move_pct = 0.003
)

# Wird sp\u00e4ter im Trainings-Notebook verwendet, um die Bewegungs-Tage zu oversamplen.
OVERSAMPLE_FACTOR = 3


In [47]:
import json
from pathlib import Path
from src.utils.io import DATA_PROCESSED

# Ordner für Experiment-Metadaten
exp_meta_dir = DATA_PROCESSED / "experiments"
exp_meta_dir.mkdir(parents=True, exist_ok=True)

# Konfiguration dieses Experiments (Label-Teil)
exp_config = {
    "exp_id": EXP_ID,
    "label_params": LABEL_PARAMS,
    # hier kannst du bei Bedarf noch weitere Dinge ergänzen,
    # z.B. OVERSAMPLE_FACTOR oder Hinweise
}

exp_config_path = exp_meta_dir / f"{EXP_ID}_config.json"
with exp_config_path.open("w", encoding="utf-8") as f:
    json.dump(exp_config, f, indent=2)

print("Experiment-Konfiguration gespeichert unter:", exp_config_path)

Experiment-Konfiguration gespeichert unter: data/processed/experiments/s3b_h4_thr0p3pct_tol0p3_30dfeat_config.json


## 2. Labels und Trainingsdatensatz automatisch erzeugen

Die folgende Zelle erledigt den kompletten Daten-Vorbereitungsschritt automatisch:

1. `label_eurusd(**LABEL_PARAMS)` berechnet die Labels im Speicher.
2. Es werden zwei Label-Dateien geschrieben:
   - `data/processed/fx/eurusd_labels__<EXP_ID>.csv` (archivierte Version f\u00fcr dieses Experiment)
   - `data/processed/fx/eurusd_labels.csv` (aktuelle Standardversion)
3. `build_training_dataframe(exp_id=EXP_ID)` erzeugt den passenden Trainingsdatensatz aus Labels + News.
4. Es werden zwei Trainingsdateien geschrieben:
   - `data/processed/datasets/eurusd_news_training__<EXP_ID>.csv` (archivierte Version)
   - `data/processed/datasets/eurusd_news_training.csv` (aktuelle Standardversion)

So muss ich im Terminal **nichts mehr manuell ausf\u00fchren**; alle relevanten CSVs entstehen beim
Start des Experiments direkt im Notebook.


In [48]:
from src.data.label_eurusd import label_eurusd
from src.data.build_training_set import build_training_dataframe
from src.utils.io import DATA_PROCESSED

# 1) Labels im Speicher berechnen (EURUSD-Zeitreihe + Lookahead-Logik).
labels = label_eurusd(**LABEL_PARAMS)

# Verzeichnis f\u00fcr Label-Dateien sicherstellen.
fx_dir = DATA_PROCESSED / "fx"
fx_dir.mkdir(parents=True, exist_ok=True)

# Archiv-Datei mit Experiment-ID, z. B.: eurusd_labels__v1_h4_thr0p5pct_strict.csv
labels_path_exp = fx_dir / f"eurusd_labels__{EXP_ID}.csv"
labels.to_csv(labels_path_exp)

# "Aktuelle" Datei ohne Suffix (f\u00fcr Default-Nutzung in anderen Skripten).
labels_path_latest = fx_dir / "eurusd_labels.csv"
labels.to_csv(labels_path_latest)

print("[ok] Labels gespeichert als:")
print("  ", labels_path_exp)
print("  ", labels_path_latest)

# 2) Trainingsdatensatz aus Labels + News bauen.
# build_training_dataframe verwendet durch exp_id automatisch die passende Label-Datei.
merged = build_training_dataframe(exp_id=EXP_ID)

ds_dir = DATA_PROCESSED / "datasets"
ds_dir.mkdir(parents=True, exist_ok=True)

# Archiv-Trainingsdatensatz mit Experiment-ID, z. B.: eurusd_news_training__v1_h4_thr0p5pct_strict.csv
train_path_exp = ds_dir / f"eurusd_news_training__{EXP_ID}.csv"
merged.to_csv(train_path_exp, index=False)

# "Aktueller" Trainingsdatensatz ohne Suffix.
train_path_latest = ds_dir / "eurusd_news_training.csv"
merged.to_csv(train_path_latest, index=False)

print("[ok] Trainingsdatensatz gespeichert als:")
print("  ", train_path_exp)
print("  ", train_path_latest)


[ok] Labels gespeichert als:
   data/processed/fx/eurusd_labels__s3b_h4_thr0p3pct_tol0p3_30dfeat.csv
   data/processed/fx/eurusd_labels.csv
[ok] Trainingsdatensatz gespeichert als:
   data/processed/datasets/eurusd_news_training__s3b_h4_thr0p3pct_tol0p3_30dfeat.csv
   data/processed/datasets/eurusd_news_training.csv


## 3. Trainingsdatensatz f\u00fcr dieses Experiment laden

Ab hier kann ich im gleichen Notebook oder in weiteren Modell-Notebooks mit dem Datensatz

```text
data/processed/datasets/eurusd_news_training__<EXP_ID>.csv
```

weiterarbeiten. Die folgende Zelle zeigt, wie der Datensatz eingelesen wird;
der Rest (Train/Val/Test-Split, XGBoost-Training, Auswertung) kann dann wie bisher erfolgen.


In [49]:
import pandas as pd

dataset_path = DATA_PROCESSED / "datasets" / f"eurusd_news_training__{EXP_ID}.csv"
print("Verwende Datensatz:", dataset_path)

df = pd.read_csv(dataset_path, parse_dates=["date"])
df.head()


Verwende Datensatz: data/processed/datasets/eurusd_news_training__s3b_h4_thr0p3pct_tol0p3_30dfeat.csv


,date,label,signal,direction,month,week,quarter,intraday_range,intraday_range_pct,body,...,cal_is_month_end,hol_is_us_federal_holiday,hol_is_day_before_us_federal_holiday,hol_is_day_after_us_federal_holiday,lookahead_return,article_count,avg_polarity,avg_neg,avg_neu,avg_pos
0,2020-04-14,neutral,0,NaN,4,16,2,0.006822,0.006246,0.000346,...,0,0,0,0,-0.005129,1,-0.949,0.0930,0.8410,0.0660
1,2020-05-28,up,1,1.0,5,22,2,0.007875,0.007148,0.000158,...,0,0,0,0,0.015665,1,0.518,0.0870,0.8200,0.0940
2,2020-07-15,up,1,1.0,7,29,3,0.005945,0.005210,-0.000013,...,0,0,0,0,0.004205,1,0.847,0.0800,0.8180,0.1020
3,2020-07-22,up,1,1.0,7,30,3,0.009239,0.008010,-0.000106,...,0,0,0,0,0.020396,2,0.925,0.0870,0.7850,0.1280
4,2020-10-19,up,1,1.0,10,43,4,0.009029,0.007707,-0.000014,...,0,0,0,0,0.008495,4,0.944,0.0615,0.8515,0.0865
